In [ ]:
!nvidia-smi

Tue Nov 12 15:40:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Downloading Necessary Libraries

In [ ]:
!pip3 install -q -U bitsandbytes
!pip3 install -q -U peft
!pip3 install -q -U trl
!pip3 install -q -U accelerate
!pip3 install -q -U datasets
!pip3 install -q -U transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 109.4 MB/s eta 0:00:00


# Hugging Face Logins

In [ ]:
import os
from google.colab import userdata
os.environ["HF_Token"] = userdata.get('HF_Token')

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

# Downloading pre-trained model and tokenizers from hugging face

In [ ]:
model_id = "google/gemma-2-2b"

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)


config.json:   0%|          | 0.00/818 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/481M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

# Inference without Training

In [ ]:
#function for inference
def get_completion(instruction: str, input_text: str, model, tokenizer) -> str:
    device = "cuda:0"

    # Adjusted prompt template
    if input_text:
        prompt_template = "<s>[INST] {instruction} {input_text} [/INST] "
        prompt = prompt_template.format(instruction=instruction, input_text=input_text)
    else:
        prompt_template = "<s>[INST] {instruction} [/INST] "
        prompt = prompt_template.format(instruction=instruction)

    # Encode the prompt
    encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
    model_inputs = encodeds.to(device)

    # Generate response
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=1000,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode the output
    decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    # Extract the model's response after the prompt
    response = decoded.replace(prompt, "").strip()

    return response


In [ ]:
instruction = "You are a helpful mental health counselling assistant, please answer the mental health questions based on the patient's description. The assistant gives helpful, comprehensive, and appropriate answers to the user's questions."
input_text ="I'm overwhelmed about my result for final semester in Masters."
response = get_completion(instruction, input_text, model, tokenizer)
print(response)


<s>[INSTRUCT] Question 1. I felt really overwhelmed about the results for my final semester in my Masters' program. I want to make a career as an engineer, and I'm quite upset that I received a low-performing result. You know what a stressful time it was for me to be the one to get that grade. It gave me a tough time. It's making me feel so upset, and I'm getting so many questions about why I failed. I don't know what to do. When I think hard, I can put forward any reasons why I failed and did it badly. I think there are many things I should have done to prepare better for the result. It was not good that I didn't do any preparation. It got me so far. It is a huge reason for failing the examination, I should never have let the deadline approach closer to when I'm doing it. It's one of the worst things that have happened to me lately. For the life of me, I'm worried, and I don't like it. I feel as if I am under a kind of psychological pressure. I am not able to solve it by my own, I nee

# Data Loading and Transformation

In [ ]:
from datasets import load_dataset, DatasetDict

# Load the dataset
dataset = load_dataset("ShenLab/MentalChat16K")

# Split the dataset into train and test
train_test_split = dataset['train'].train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']


train_valid_split = train_dataset.train_test_split(test_size=0.1)
train_dataset = train_valid_split["train"]
validation_dataset = train_valid_split["test"]

# Combine into a DatasetDict
split_dataset = DatasetDict({
    "train": train_dataset,
    "validation": validation_dataset,
    "test": test_dataset
})

print(split_dataset)
dataset = split_dataset


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.58k [00:00<?, ?B/s]

Interview_Data_6K.csv:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

Synthetic_Data_10K.csv:   0%|          | 0.00/32.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16084 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 11580
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 1287
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 3217
    })
})


In [ ]:
#function to transform prompt
def transform_example(example):

    instruction = example['instruction'].strip()
    input_text = example['input'].strip() if example['input'] else ""
    output_text = example['output'].strip()


    if input_text:
        formatted_text = f"<s>[INST] {instruction} {input_text} [/INST] {output_text} </s>" #for cases when ther's no INST
    else:
        formatted_text = f"<s>[INST] {instruction} [/INST] {output_text} </s>"

    return {'text': formatted_text}

# transformer function call
transformed_dataset = DatasetDict({
    split: data.map(transform_example, remove_columns=data.column_names)
    for split, data in dataset.items()
})

print(transformed_dataset['train'][0]['text'])

Map:   0%|          | 0/11580 [00:00<?, ? examples/s]

Map:   0%|          | 0/1287 [00:00<?, ? examples/s]

Map:   0%|          | 0/3217 [00:00<?, ? examples/s]

<s>[INST] You are a helpful mental health counselling assistant, please answer the mental health questions based on the patient's description. 
The assistant gives helpful, comprehensive, and appropriate answers to the user's questions. I've been feeling increasingly anxious and on edge lately. I can't seem to focus on anything, and my heart races whenever I think about the future or upcoming events. I've tried to calm myself down with deep breathing exercises, but they don't seem to help. [/INST] Anxiety can be a debilitating experience, and it's essential to remember that you're not alone in feeling this way. I'd like to help you explore some evidence-based interventions for managing anxiety, such as cognitive-behavioral techniques, relaxation exercises, or mindfulness practices. We can also work together to identify any specific triggers or patterns that might be contributing to your anxiety, and develop a plan for managing these triggers effectively. Additionally, we can explore th

In [ ]:
transformed_dataset['train']

Dataset({
    features: ['text'],
    num_rows: 11580
})

# Defining Q-LoRa Parameters

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
import bitsandbytes as bnb
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=['v_proj', 'up_proj', 'o_proj', 'q_proj', 'down_proj', 'gate_proj', 'k_proj'],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

# Training Pre-trained model for 1 epoch

In [ ]:
import transformers

from trl import SFTTrainer


tokenizer.pad_token = tokenizer.eos_token
torch.cuda.empty_cache()


trainer = SFTTrainer(
    model=model,
    train_dataset=transformed_dataset['train'],
    eval_dataset=transformed_dataset['validation'],
    dataset_text_field="text",
    peft_config=lora_config,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        #warmup_steps=0.03,
        max_steps=-1,
        learning_rate=2e-4,
        logging_steps=20,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
        report_to="tensorboard",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:403: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due 

In [ ]:
os.environ["WANDB_MODE"] = "disabled"

In [ ]:
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
20,0.814900
40,0.826800
60,0.860800
80,0.924100
100,0.982700


TrainOutput(global_step=100, training_loss=0.8818642997741699, metrics={'train_runtime': 250.0871, 'train_samples_per_second': 1.599, 'train_steps_per_second': 0.4, 'total_flos': 2718050911220736.0, 'train_loss': 0.8818642997741699, 'epoch': 0.03454231433506045})

# Saving Model in hugging Face Repo

In [ ]:
trainer.model.save_pretrained(new_model)

In [ ]:
new_model = "gemma2-MentalHealth-Finetune-test" #Name of the model pushing to huggingface

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)

merged_model= PeftModel.from_pretrained(base_model, new_model)
merged_model= merged_model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# Push the model and tokenizer to the Hugging Face
merged_model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/DiatWork/gemma2-MentalHealth-Finetune-test/commit/452ff4cae8b47e4e594c689c0ccf7e980e2a03b5', commit_message='Upload tokenizer', commit_description='', oid='452ff4cae8b47e4e594c689c0ccf7e980e2a03b5', pr_url=None, pr_revision=None, pr_num=None)

# Saving in Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
save_path = "/content/drive/MyDrive/gemma2-MentalHealth-Finetune"

# directory
os.makedirs(save_path, exist_ok=True)

# Save the model and tokenizer to Google Drive
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)


('/content/drive/MyDrive/gemma2-MentalHealth-Finetune/tokenizer_config.json',
 '/content/drive/MyDrive/gemma2-MentalHealth-Finetune/special_tokens_map.json',
 '/content/drive/MyDrive/gemma2-MentalHealth-Finetune/tokenizer.model',
 '/content/drive/MyDrive/gemma2-MentalHealth-Finetune/added_tokens.json',
 '/content/drive/MyDrive/gemma2-MentalHealth-Finetune/tokenizer.json')

# Loading Fine-Tuned Model from Hugging Face Repo

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import logging
from transformers import logging as transformers_logging
import warnings

logging.basicConfig(level=logging.WARNING)
transformers_logging.set_verbosity_warning()
warnings.filterwarnings("ignore", category=UserWarning)

# Load the model and tokenizer
model_id = "DiatWork/gemma2-MentalHealth-Finetune-test"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")
model.config.use_cache = False  # Disable cache

# Inference

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"


In [ ]:
import torch
import textwrap

# Input with Prompt
input_text = "<s>[INST] You are a helpful mental health assistant. I'm very stressed because I have a Job Interview, though I have practiced for it but this is my first interview so I'm anxious. How to deal with this stress? [/INST] "

inputs = tokenizer(input_text, return_tensors="pt").to(model.device). # Tokenize the input

# Generate response
with torch.no_grad():
    output = model.generate(**inputs, max_new_tokens=400)

decoded_output = tokenizer.decode(output[0], skip_special_tokens=True).strip() # Tokenize the output

# Presentation
if "[INST]" in decoded_output and "[/INST]" in decoded_output:
    question, model_answer = decoded_output.split("[/INST]", 1)
else:
    question, model_answer = decoded_output, ""  # Handle cases where split fails


question = "\n".join(textwrap.wrap(question, width=80))
model_answer = "\n".join(textwrap.wrap(model_answer.strip(), width=80))

print("Question:\n", question)
print("\nModel Answer:\n", model_answer)
print("\n" + "=" * 80 + "\n")


Question:
 <s>[INST] You are a helpful mental health assistant. I'm very stressed because I
have a Job Interview, though I have practiced for it but this is my first
interview so I'm anxious. How to deal with this stress?

Model Answer:
 It's great that you've been practicing for your job interview and that you're
feeling anxious, but it's important to remember that everyone feels some level
of stress before an important event. Here are some tips to help you manage your
stress and feel more confident during your interview:  1. Prepare thoroughly:
Make sure you have a clear understanding of the job description and the company
culture. Research the company and its products or services to show your
knowledge and interest.  2. Practice your answers: Rehearse your responses to
common interview questions. This will help you feel more comfortable and
confident when answering questions during the interview.  3. Dress
appropriately: Choose a professional outfit that makes you feel confident and

In [ ]:
# Input with Prompt
input_text = "<s>[INST] You are a helpful mental health assistant. I'm very stressed because I have a group project in last semester of my college and I want to get first honours, I'm scared. Please help me cope with stress. [/INST]"

inputs = tokenizer(input_text, return_tensors="pt").to(model.device). # Tokenize the input

# Generate response
with torch.no_grad():
    output = model.generate(**inputs, max_new_tokens=200)

decoded_output = tokenizer.decode(output[0], skip_special_tokens=True).strip() # Tokenize the output

# Presentation
if "[INST]" in decoded_output and "[/INST]" in decoded_output:
    question, model_answer = decoded_output.split("[/INST]", 1)
else:
    question, model_answer = decoded_output, ""  # Handle cases where split fails


question = "\n".join(textwrap.wrap(question, width=80))
model_answer = "\n".join(textwrap.wrap(model_answer.strip(), width=80))

print("Question:\n", question)
print("\nModel Answer:\n", model_answer)
print("\n" + "=" * 80 + "\n")



Question:
 <s>[INST] You are a helpful mental health assistant. I'm very stressed because I
have a group project in last semester of my college and I want to get first
honours, I'm scared. Please help me cope with stress.

Model Answer:
 It's understandable that you're feeling stressed about your group project in
college. The pressure to perform well and achieve first honours can be
overwhelming, especially during your final semester. However, there are steps
you can take to manage your stress and increase your chances of success.  First,
it's important to prioritize your time and focus on the project at hand. Break
down the tasks into smaller, manageable steps and set realistic deadlines for
yourself. This will help you stay organized and focused on the task at hand.
Additionally, consider reaching out to your group members and discussing your
concerns. Open communication can help alleviate any misunderstandings or
conflicts that may be contributing to your stress.  Taking care of you

# Evaluation

In [ ]:
!pip install evaluate bert_score transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.4 MB/s eta 0:00:00


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 11580
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 1287
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 3217
    })
})

# Validation set Response under max_length=100

In [ ]:
import textwrap
import torch
import evaluate

# validation data for evaluation
test_data = dataset["test"]
subset_size = len(test_data)

prompts = []
references = []

# Build prompts and references
for i in range(subset_size):
    sample = test_data[i]
    instruction = sample['instruction']
    input_text = sample['input']
    output_text = sample['output']

    if input_text:
        prompt = f"<s>[INST] {instruction} {input_text} [/INST]"
    else:
        prompt = f"<s>[INST] {instruction} [/INST]"

    prompts.append(prompt)
    references.append(output_text)  # reference answer

# Generate responses for each prompt and format them
generated_texts = []
for prompt in prompts:
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=300,
            do_sample=True,
            top_p=0.9,
            top_k=50,
            temperature=0.7,
            use_cache=False,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode the response
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

    # Presentation
    try:
        question, model_answer = decoded_output.split("[/INST]", 1)
    except ValueError:
        question, model_answer = decoded_output, ""


    question = "\n".join(textwrap.wrap(question, width=80))
    model_answer = "\n".join(textwrap.wrap(model_answer, width=80))

    print("Question:\n", question)
    print("\nModel Answer:\n", model_answer)
    print("\n" + "="*80 + "\n")

    # Append decoded output to generated_texts for evaluation
    generated_texts.append(decoded_output)




Streaming output truncated to the last 5000 lines.
definitely not an easy thing to deal with. It's important to acknowledge that
you're not alone in this and that many people struggle with anxiety at some
point in their lives. One thing that could be helpful for you is to start
exploring the underlying causes of your anxiety. This could involve looking at
any specific triggers or situations that seem to make your anxiety worse. For
example, are there any particular social situations or


Question:
 <s>[INST] You are a helpful mental health counselling assistant, please answer
the mental health questions based on the patient's description.  The assistant
gives helpful, comprehensive, and appropriate answers to the user's questions.
[Name], a retired financial advisor, expresses deep passion for his areas of
expertise - financing, money, and real estate. His eyes sparkle, and his voice
gains energy when he discusses these topics. However, I'm worried that involving
helpers in these conve

In [ ]:

!pip install rouge_score
# Initialize evaluation metrics
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")

# Calculate Scores
bleu_score = bleu.compute(predictions=generated_texts, references=[[ref] for ref in references])
rouge_score = rouge.compute(predictions=generated_texts, references=references)
bert_score = bertscore.compute(predictions=generated_texts, references=references, lang="en")

# Print Results
print("BLEU Score:", bleu_score)
print("ROUGE Score:", rouge_score)
print("BERTScore (Precision, Recall, F1):", bert_score["precision"], bert_score["recall"], bert_score["f1"])




  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=6eda24cffec7cca993ea620c7553d771705c66dc40f354073560721e3801602b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BLEU Score: {'bleu': 0.07808172642945549, 'precisions': [0.4846667367033723, 0.15515141635181526, 0.06921536343241187, 0.038687253461211234], 'brevity_penalty': 0.6554764106557096, 'length_ratio': 0.7030405983838534, 'translation_length': 209414, 'reference_length': 297869}
ROUGE Score: {'rouge1': 0.39491371534801056, 'rouge2': 0.1251342804310105, 'rougeL': 0.17569518549589863, 'rougeLsum': 0.30731553918793153}
BERTScore (Precision, Recall, F1): [0.8505979180335999, 0.8670139908790588, 0.835503339767456, 0.8707054257392883, 0.865147590637207, 0.8706592321395874, 0.8524398803710938, 0.8561485409736633, 0.8553181886672974, 0.8403328657150269, 0.8776812553405762, 0.8597607612609863, 0.8516089916229248, 0.8632280230522156, 0.8495675325393677, 0.8579204082489014, 0.848534882068634, 0.865302562713623, 0.8476221561431885, 0.8624709248542786, 0.866837203502655, 0.8941774368286133, 0.8534433841705322, 0.8509887456893921, 0.8550397157669067, 0.8341479301452637, 0.8583222031593323, 0.874354720115

# Testing on 50% Test Dataset

In [ ]:
!pip install json

ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [ ]:
import textwrap
import torch
import evaluate

# 50% test data for evaluation
test_data = dataset["test"]
subset_size = int(0.5 * len(test_data))

prompts = []
references = []

# Build prompts and references
for i in range(subset_size):
    sample = test_data[i]
    instruction = sample['instruction']
    input_text = sample['input']
    output_text = sample['output']

    if input_text:
        prompt = f"<s>[INST] {instruction} {input_text} [/INST]"
    else:
        prompt = f"<s>[INST] {instruction} [/INST]"

    prompts.append(prompt)
    references.append(output_text)  # reference answer

# Generate responses for each prompt and format them
generated_texts = []
for prompt in prompts:
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=300,
            do_sample=True,
            top_p=0.9,
            top_k=50,
            temperature=0.7,
            use_cache=False,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode the response
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

    # Presentation
    try:
        question, model_answer = decoded_output.split("[/INST]", 1)
    except ValueError:
        question, model_answer = decoded_output, ""


    question = "\n".join(textwrap.wrap(question, width=80))
    model_answer = "\n".join(textwrap.wrap(model_answer, width=80))

    print("Question:\n", question)
    print("\nModel Answer:\n", model_answer)
    print("\n" + "="*80 + "\n")

    # Append decoded output to generated_texts for evaluation
    generated_texts.append(decoded_output)



!pip install rouge_score
# Initialize evaluation metrics
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")

# Calculate Scores
bleu_score = bleu.compute(predictions=generated_texts, references=[[ref] for ref in references])
rouge_score = rouge.compute(predictions=generated_texts, references=references)
bert_score = bertscore.compute(predictions=generated_texts, references=references, lang="en")

# Print Results
print("BLEU Score:", bleu_score)
print("ROUGE Score:", rouge_score)
print("BERTScore (Precision, Recall, F1):", bert_score["precision"], bert_score["recall"], bert_score["f1"])


from huggingface_hub import Repository

repo = Repository(local_dir="./evaluation_results", clone_from="DiatWork/GPT-Neox-MentalHealth-Finetune-10epoch")

# Save metrics to JSON file
evaluation_metrics = {
    "ROUGE": rouge_score,
    "BERTScore": bert_score,
    "BLEU": bleu_score,
}

import json

output_file = "evaluation_metrics.json"
with open(output_file, "w") as json_file:
    json.dump(evaluation_metrics, json_file, indent=4)



Streaming output truncated to the last 5000 lines.
helpful in managing anxiety. Are there specific therapeutic techniques or
exercises that I can incorporate into my daily routine to find relief from
anxious thoughts? How important is self-care as a part of anxiety management?
Instruction 10: Doctor, I have been dealing with an overwhelming amount of
stress in various aspects of my life, and it's starting to take its toll on my
emotional well-being. My goal for this counseling session is to explore
effective strategies to reduce stress levels and regain control over my life.
One potential trigger for my stress is the constant pressure to meet
expectations at work. The demand for high productivity paired with tight
deadlines creates an environment where I feel constantly overwhelmed. It seems
impossible to keep up with the workload while maintaining a healthy work-life
balance.  For example, just last week, I had multiple projects overlapping, all
with strict deadlines. Having to priori

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Download file model-00002-of-00002.safetensors:   0%|          | 32.0k/230M [00:00<?, ?B/s]

Clean file tokenizer.model:   0%|          | 1.00k/4.04M [00:00<?, ?B/s]

Clean file tokenizer.json:   0%|          | 1.00k/32.8M [00:00<?, ?B/s]

Clean file model-00002-of-00002.safetensors:   0%|          | 1.00k/230M [00:00<?, ?B/s]

Clean file model-00001-of-00002.safetensors:   0%|          | 1.00k/4.65G [00:00<?, ?B/s]

# Evaluation Results: BERT Avg.

In [ ]:
import json
import numpy as np

file_path = '/content/evaluation_metrics.json'

with open(file_path, 'r') as file:
    gemma_data = json.load(file)

# Calculate Avg. BERTScore
bert_score = gemma_data["BERTScore"]

precision_avg = np.mean(bert_score["precision"])
recall_avg = np.mean(bert_score["recall"])
f1_avg = np.mean(bert_score["f1"])

precision_avg, recall_avg, f1_avg

(0.8590813238760829, 0.854886392276777, 0.8569042132736031)